# A14_Workshop_and_Tools - Trade Performance Analyzer


In [ ]:
## Read Trade Transactions File

In [3]:
import pandas as pd
import plotly.express as px
import numpy as np

def read_trades(file_path, file_type='xlsx'):
    if file_type == 'xlsx':
        df = pd.read_excel(file_path)
    elif file_type == 'csv':
        df = pd.read_csv(file_path)
    else:
        raise ValueError("Unsupported file type. Please use 'xlsx' or 'csv'.")
    
    return df

file_path = r"C:\Users\PowerUser\Dropbox\! 0 - AA TOSrecords\! - 0 - Aeromir\A14_Workshop_and_Tools\A14-Class-Trade-Transaction-Performance-History-v2025.10.22.xlsx"

if __name__ == "__main__":
    #file_path = "path/to/your/file.xlsx"  # Replace with your file path
    file_path = r"C:\Users\PowerUser\Dropbox\! 0 - AA TOSrecords\! - 0 - Aeromir\A14_Workshop_and_Tools\A14-Class-Trade-Transaction-Performance-History-v2025.10.22.xlsx"
    
    trade_df = read_trades(file_path)

    trade_df

In [ ]:
trade_df

In [ ]:
## Show Columns

trade_data_columns = trade_df.columns
trade_data_columns

In [ ]:

def analyze_trades(df):
    # Calculate profit/loss for each trade
    df['P/L'] = df['Exit Price'] - df['Entry Price']
    
    # Calculate cumulative profit/loss
    df['Cumulative P/L'] = df['P/L'].cumsum()
    
    # Calculate win rate
    win_rate = (df['P/L'] > 0).sum() / len(df)
    
    # Calculate average profit and loss
    avg_profit = df[df['P/L'] > 0]['P/L'].mean()
    avg_loss = df[df['P/L'] < 0]['P/L'].mean()
    
    # Calculate maximum drawdown
    max_drawdown = (df['Cumulative P/L'].cummax() - df['Cumulative P/L']).max()
    
    return {
        'win_rate': win_rate,
        'avg_profit': avg_profit,
        'avg_loss': avg_loss,
        'max_drawdown': max_drawdown
    }


In [ ]:

def create_plot(df):
    fig = px.line(df, x='Date', y='Cumulative P/L', title='Cumulative Profit/Loss Over Time')
    fig.update_xaxes(title='Date')
    fig.update_yaxes(title='Cumulative Profit/Loss')
    return fig

def main(file_path, file_type='xlsx'):
    df = read_trades(file_path, file_type)
    
    # Ensure required columns exist
    required_columns = ['Date', 'Entry Price', 'Exit Price']
    if not all(col in df.columns for col in required_columns):
        raise ValueError("The file is missing required columns: Date, Entry Price, and Exit Price.")
    
    # Convert Date column to datetime
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Sort by date
    df = df.sort_values('Date' ascending=False)
    
    performance = analyze_trades(df)
    fig = create_plot(df)
    
    print("Trade Performance Analysis:")
    print(f"Win Rate: {performance['win_rate']:.2%}")
    print(f"Average Profit: ${performance['avg_profit']:.2f}")
    print(f"Average Loss: ${performance['avg_loss']:.2f}")
    print(f"Maximum Drawdown: ${performance['max_drawdown']:.2f}")
    
    fig.show()

if __name__ == "__main__":
    #file_path = "path/to/your/file.xlsx"  # Replace with your file path
    file_path = r"C:\Users\PowerUser\Dropbox\! 0 - AA TOSrecords\! - 0 - Aeromir\A14_Workshop_and_Tools\A14-Class-Trade-Transaction-Performance-History-v2025.10.22.xlsx"
    
    main(file_path)

### Column Names Map
- Trade = Trade_ID
- Opened = Date_Opened
- Closed = Date_Closed
- DIT = Days_In_Trade
- Description = Description
- Symbol = Ticker_Symbol
- Planned Capital = Planned_Capital
- Maximum Margin = Margin_Max
- Current Margin = Margin_Current
- Profit/Loss = PnL
- Yield on Max Margin = Yield_on_Max_Margin
- Yield on Planned Capital = Yield_on_Planned_Capital
- Short Strike Delta = Short_Strike_Delta
- Position Delta = Position_Delta
- Updated (Eastern) = Updated_Eastern
- View Details = View_Details


In [ ]:
import streamlit as st
import pandas as pd
import plotly.express as px
import numpy as np

def clean_column_names(df):
    return df.rename(columns=lambda x: x.replace(' ', '_').replace('/', '_').replace('(', '').replace(')', ''))

def analyze_trades(df, trade_col, opened_col, max_margin_col, profit_loss_col):
    df['Cumulative_Profit_Loss'] = df[profit_loss_col].cumsum()
    return df

def create_plot(df, profit_loss_col):
    fig = px.line(df, x='Opened', y='Cumulative_Profit_Loss', title='Cumulative Profit/Loss Over Time')
    fig.update_xaxes(title='Date')
    fig.update_yaxes(title='Cumulative Profit/Loss')
    return fig

def main():
    st.title("Trade Transaction Performance Analyzer")

    # File upload
    file = st.file_uploader("Upload your XLSX or CSV file", type=["xlsx", "csv"])
    if not file:
        st.stop()

    # Read the file
    if file.name.endswith('.xlsx'):
        df = pd.read_excel(file)
    else:
        df = pd.read_csv(file)

    # Clean column names
    df = clean_column_names(df)

    # Display column names
    st.subheader("Available columns:")
    st.write(df.columns.tolist())

    # Get user input for column selection
    st.subheader("Select columns for analysis:")
    trade_col = st.selectbox("Trade column", df.columns)
    opened_col = st.selectbox("Opened column", df.columns)
    max_margin_col = st.selectbox("Maximum Margin column", df.columns)
    profit_loss_col = st.selectbox("Profit/Loss column", df.columns)

    # Analyze trades
    df = analyze_trades(df, trade_col, opened_col, max_margin_col, profit_loss_col)

    # Sort by date
    df = df.sort_values('Closed', ascending=False)

    # Display data
    st.subheader("Trade data:")
    st.dataframe(df)

    # Create and display plot
    st.subheader("Cumulative Profit/Loss Chart:")
    fig = create_plot(df, profit_loss_col)
    st.plotly_chart(fig, use_container_width=True)

    # Allow user to select date range for chart
    st.subheader("Select date range for chart:")
    start_date = st.date_input("Start date", value=df['Opened'].min())
    end_date = st.date_input("End date", value=df['Opened'].max())

    # Filter data and create new plot
    filtered_df = df[(df['Opened'] >= start_date) & (df['Opened'] <= end_date)]
    fig = create_plot(filtered_df, profit_loss_col)
    st.subheader("Filtered Cumulative Profit/Loss Chart:")
    st.plotly_chart(fig, use_container_width=True)

if __name__ == "__main__":
    main()

In [ ]:
import streamlit as st
import pandas as pd
import plotly.express as px
import numpy as np

def clean_column_names(df):
    return df.rename(columns=lambda x: x.replace(' ', '_').replace('/', '_').replace('(', '').replace(')', ''))

def analyze_trades(df, trade_col, opened_col, max_margin_col, profit_loss_col):
    try:
        df['Cumulative_Profit_Loss'] = df[profit_loss_col].cumsum()
        return df
    except Exception as e:
        st.error(f"Error analyzing trades: {str(e)}")
        return None

def create_plot(df, profit_loss_col):
    try:
        fig = px.line(df, x='Opened', y='Cumulative_Profit_Loss', title='Cumulative Profit/Loss Over Time')
        fig.update_xaxes(title='Date')
        fig.update_yaxes(title='Cumulative Profit/Loss')
        return fig
    except Exception as e:
        st.error(f"Error creating plot: {str(e)}")
        return None

def main():
    st.title("Trade Transaction Performance Analyzer")

    # File upload
    file = st.file_uploader("Upload your XLSX or CSV file", type=["xlsx", "csv"])
    if not file:
        st.stop()

    # Read the file
    try:
        if file.name.endswith('.xlsx'):
            df = pd.read_excel(file)
        else:
            df = pd.read_csv(file)
    except Exception as e:
        st.error(f"Error reading file: {str(e)}")
        return

    # Clean column names
    try:
        df = clean_column_names(df)
    except Exception as e:
        st.error(f"Error cleaning column names: {str(e)}")
        return

    # Sort by date
    try:
        df = df.sort_values('Date', ascending=False)
    except Exception as e:
        st.error(f"Error sorting by date: {str(e)}")
        return

    # Display column names
    st.subheader("Available columns:")
    st.write(df.columns.tolist())

    # Get user input for column selection
    st.subheader("Select columns for analysis:")
    trade_col = st.selectbox("Trade column", df.columns)
    opened_col = st.selectbox("Opened column", df.columns)
    max_margin_col = st.selectbox("Maximum Margin column", df.columns)
    profit_loss_col = st.selectbox("Profit/Loss column", df.columns)

    # Analyze trades
    df = analyze_trades(df, trade_col, opened_col, max_margin_col, profit_loss_col)
    if df is None:
        return

    # Display data
    st.subheader("Trade data:")
    st.dataframe(df)

    # Create and display plot
    st.subheader("Cumulative Profit/Loss Chart:")
    fig = create_plot(df, profit_loss_col)
    if fig is not None:
        st.plotly_chart(fig, use_container_width=True)

    # Allow user to select date range for chart
    st.subheader("Select date range for chart:")
    try:
        start_date = st.date_input("Start date", value=df['Opened'].min())
        end_date = st.date_input("End date", value=df['Opened'].max())
    except Exception as e:
        st.error(f"Error selecting date range: {str(e)}")
        return

    # Filter data and create new plot
    try:
        filtered_df = df[(df['Opened'] >= start_date) & (df['Opened'] <= end_date)]
        fig = create_plot(filtered_df, profit_loss_col)
        if fig is not None:
            st.subheader("Filtered Cumulative Profit/Loss Chart:")
            st.plotly_chart(fig, use_container_width=True)
    except Exception as e:
        st.error(f"Error filtering data or creating filtered plot: {str(e)}")

if __name__ == "__main__":
    main()